In [1]:
from fh import FH 
import fh
from treeBuilder import TreeBuilder as TB
import PydalChanel as pydal
import copy
import pickle
import arpeggiator

lp = FH()
lp.addForwardingHandler(Arpeggiator())

oneHitShift = fh.oneHitShift

read = pydal.read
end = pydal.end
tempo = pydal.tempo
rootScale = lp.rootScale
stripSymbols = lambda beats : map(lambda beat : list(beat[1])[0], beats)

ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)
ch3 = pydal.newChannel(3)
ch4 = pydal.newChannel(4)
ch5 = pydal.newChannel(5)

idv = lambda a : a
codec = lambda buf: fh.noteListToHitList(fh.flattenByBeat(fh.notesByBeat(fh.hitListToNoteList(buf))))

NameError: name 'Arpeggiator' is not defined

In [ ]:
lp.buttonForwardingHandlers

In [ ]:
a = read("{bd (lt*2, lt, lt), hc <~, sn hc> ~ hc:2}")
b = read("{(bd ~, sn:3 <~, cp>)}")
c = read("((bd:2*2, [bd:3 bd:3] bd), [sn:5 [bd sn:3]], [[bd sn:3] bd], <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>)")
c2 = read("(bd:2*2, [bd:3 bd:3] bd) [sn:5 [bd sn:3]] [[bd sn:3] bd] <sn:6 [bd sn:3], <sn:6*4, sn:6*2 ~, sn:6*2 [sn:6 ~]>>", 4)
d = read("{~ ~, bin ~ bin ~}")
e = read("{[bin <bin:1, bin, bin*2> bin bot, hc <hc, ho>], bd sn:3}")


demo = read("bd ~")

f = read("aa:1_1.75*2 aa:3_3.75*3", 2, 'sample')
d2 = read("{~ ~ , bd ~ sn:3 ~}", 4)
g = read("{~ ~ , ab:0_2 ~ ab:3_5 ~}", 4, 'sample')

In [ ]:
ch1.stop()

In [ ]:
tempo(150)

In [ ]:
lp.loadScenesFromFile("song1e.scenes")

In [ ]:
rootScale(0, lp.roots[0]+5, "hminor")
rootScale(1, lp.roots[1]+5, "hminor")
rootScale(2, lp.roots[2]+5, "hminor")

In [ ]:
lp.saveScenesToFile("song1slow.scenes")

In [ ]:
st = '69 0.49950693402567,80,101,1,on-0.12300503053356,80,0,1,off-0.37064133602195,75,98,1,on-0.10712906254986,75,0,1,off-0.36297831440561,75,103,1,on-0.090360763184094,75,0,1,off-0.40264527718247,75,108,1,on-0.098848608124257,75,0,1,off-0.39581608216994,75,103,1,on-0.074119008398242,75,0,1,off-0.41079760461554,75,111,1,on-0.073419676461185,75,0,1,off-0.41209621156585,75,112,1,on-0.12404576558649,75,0,1,off-0.45459032517527,0,0,0,timeAfterLastHit 216'
for s in st.split('-'):
    print s

In [ ]:
lp.startChannel(1, lambda buf: fh.randTranspose(buf, lp.roots[2], lp.scales[2]), lp.loops[2][0])

In [ ]:
lp.stopChannel(1)